In [1]:
import torch
import os

os.chdir(r"C:\Users\cunn2\OneDrive\DSML\Project\thesis-repo")

rel_hnsw_10k = torch.load(r"rel_IndexHNSWFlat_10k_results.pt")




C:\Users\cunn2\AppData\Local\Temp\ipykernel_313844\764491351.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rel_hnsw_10k = torch.load(r"rel_IndexHNSWFlat_10k_results.pt

In [3]:
rel_hnsw_10k.keys()

dict_keys(['model', 'dataset_size', 'index_type', 'index_params', 'metrics'])

In [5]:
rel_hnsw_10k['metrics'].keys()


dict_keys(['chunk_transposed', 'chunk_one_pitch_shifted', 'chunk_note_duration_changed', 'chunk_note_deleted', 'chunk_note_inserted', 'query_times', 'memory_used_MB'])